In [38]:
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import pandas as pd
from datetime import datetime
import numpy as np

df = pd.read_csv('stations_data.csv', sep=',')
def map_to_five(x):
    return min(x, 5)
df['rounded_last_update_5mins'] = pd.to_datetime(df['rounded_last_update_5mins'])
df['time_as_fraction'] = df['rounded_last_update_5mins'].dt.hour * 3600 + df['rounded_last_update_5mins'].dt.minute * 60 + df['rounded_last_update_5mins'].dt.second
weekday=df['rounded_last_update_5mins'].dt.dayofweek
weekdaydf = pd.get_dummies(weekday, prefix='day_of_week')
df = pd.concat([df, weekdaydf], axis=1)



In [51]:

# Step 2: Convert these seconds to a fraction of the day
df['time_as_fraction'] = df['time_as_fraction'] / 86400
# Applying the function to the 'available_bike_stands' column
#y_set = pd.DataFrame(df['available_bike_stands'].apply(map_to_five))
y_set = df['available_bike_stands']
y_set =  pd.DataFrame(y_set)
# Ensure x_set is in the correct shape
x_set = df[['number','time_as_fraction','day_of_week_0','day_of_week_1','day_of_week_2','day_of_week_3','day_of_week_4','day_of_week_5','day_of_week_6','temp','feels_like']]
#x_set =df.select_dtypes(include=['number'])
print(df.columns)
#x_set = df[['number','time_as_fraction','temp','feels_like']]
#x_set = df.drop(['available_bike_stands',"name","weather_desc","weather_brief",], axis=1)



# For the remainder dataset, separate X and Y as you did initially


Index(['number', 'name', 'position_lat', 'position_lng', 'available_bikes',
       'available_bike_stands', 'rounded_last_update',
       'rounded_last_update_5mins', 'temp', 'feels_like', 'humidity',
       'rain_1h', 'weather_desc', 'weather_brief', 'wind_speed', 'weatherid',
       'time_as_fraction', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2',
       'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6'],
      dtype='object')


In [52]:
station_number = x_set['number'].unique()
print(station_number)
x_set.columns

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  71  72  73  74
  75  76  77  78  79  80  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117]


Index(['number', 'time_as_fraction', 'day_of_week_0', 'day_of_week_1',
       'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
       'day_of_week_6', 'temp', 'feels_like'],
      dtype='object')

In [53]:
print(y_set)

        available_bike_stands
0                          30
1                          30
2                          30
3                          29
4                          29
...                       ...
622716                     40
622717                     40
622718                     40
622719                     40
622720                     40

[622721 rows x 1 columns]


In [54]:
maxv = y_set.max()

In [60]:
for num in station_number:
    x_set_num = x_set.loc[x_set['number'] == num]
    y_set_num = y_set[x_set['number'] == num]
    
    # Drop 'number' column correctly
    x_set_num = x_set_num.drop('number', axis=1)
    
    # One-hot encode correctly
    y_set_num = to_categorical(y_set_num, num_classes=41)  # Ensure num_classes matches your model's output
    
    # Convert to DataFrame if necessary (may not be needed if using TensorFlow/Keras directly)
    # y_set_num = pd.DataFrame(y_set_num)
    
    train_x = x_set_num.sample(frac=.8, replace=True)
    remainder_x = x_set_num.drop(train_x.index)

    # Ensure that y_set is correctly indexed and matched
    y_train = y_set_num[train_x.index]
    remainder_y = y_set_num[remainder_x.index]
    
    # Define model as before...
    model = tf.keras.Sequential([
        tf.keras.layers.Normalization(input_shape=[10,], axis=None),
        tf.keras.layers.Dense(1000, activation='relu'),
        tf.keras.layers.Dense(41, activation='softmax', use_bias=True)
    ])
    
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy', 'mse', 'categorical_accuracy'])
    
    # Fit model using correctly shaped data
    model.fit(train_x, y_train, epochs=20, batch_size=5)
    
    # Evaluate model using correctly shaped data
    evaluation = model.evaluate(remainder_x, remainder_y, verbose=2)
    print(f"Model evaluation on remainder set: {evaluation}")


Epoch 1/20


c:\Users\thede\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.0377 - categorical_accuracy: 0.0377 - loss: 9.1687 - mse: 0.0296
Epoch 2/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 684us/step - accuracy: 0.0698 - categorical_accuracy: 0.0698 - loss: 4.4218 - mse: 0.0251
Epoch 3/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 694us/step - accuracy: 0.0800 - categorical_accuracy: 0.0800 - loss: 3.5030 - mse: 0.0237
Epoch 4/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 726us/step - accuracy: 0.0974 - categorical_accuracy: 0.0974 - loss: 3.3711 - mse: 0.0234
Epoch 5/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 701us/step - accuracy: 0.1064 - categorical_accuracy: 0.1064 - loss: 3.2889 - mse: 0.0233
Epoch 6/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 706us/step - accuracy: 0.1129 - categorical_accuracy: 0.1129 - loss: 3.2568 - mse: 0.0232
Epoch 7/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step - accuracy: 0.1233 - categorical_accuracy: 0.1233 - loss: 3.1901 - mse: 0.0231
Epoch 8/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 1s 680us/step - accuracy: 0.1124 - categ

IndexError: index 6807 is out of bounds for axis 0 with size 5164